# Not for demo. But we used this notebook to do an accurate B-spline.

## B-splineデモ 02

Noshita, Koji <noshita@morphometrics.jp>

できるだけシンプルなB-spline fittingによるcurve fragmentの後処理のデモンストレーション

切れ込みがあるもの，セレーションのあるものへの適用

### 方針

* [make_lsq_spline](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.make_lsq_spline.html)を利用する
* 周期性は仮定しない
* （曲線上の位置を示す）パラメータへは微小ノイズを与えて同じ値が含まれないようにするハックをおこなう

In [ ]:
import glob, pickle

import numpy as np
import scipy as sp
import pandas as pd

import open3d as o3d

from scipy.interpolate import make_lsq_spline, make_smoothing_spline

from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
def cvt_polar(x,y):
    """Convert to polar coordinate system
    x, y -> r, theta
    """
    r = np.sqrt(x**2+y**2)
    theta = np.sign(y)*np.arccos(x/np.sqrt(x**2+y**2))
    return r, theta

def generate_knots(start, end, n_interval, d):
    """Generate knot vector
    Parameters
    ==============
    start: float
    end: float
        start and end of knots
    n_interval: int
        number of interval
    d: int
        degree of B-spline
    
    Returns
    ==============
    knots: array, shape(n_interval+2*dim + 1)
        knot vector
    
    """
    step = (end-start)/n_interval
    knots_ = [i for i in np.arange(start, end+step, step)]
    knots_p = [knots_[0] for i in range(d)]
    knots_a = [knots_[-1] for i in range(d)]
    
    knots = np.array(knots_p + knots_ + knots_a)
    return knots

## データ読み込み・確認

In [ ]:
# FILE_PATH = "../data/Bspline_202311/fix_45_maple_supported_curves.pickle"
name = "rolling_alpha_004"
FILE_PATH = "data/supported_curve/55_"+name+"_supported_curves.pickle"


with open(FILE_PATH, "rb") as f:
    curve_fragments = pickle.load(f)
    curve_fragments = [curve for curves in curve_fragments for i, curve in enumerate(curves) if i%50 == 0]
len(curve_fragments)

In [ ]:
coord_3d = np.concatenate(curve_fragments)

In [ ]:
x,y,z = coord_3d[0:-1:100].T

fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=1,
        color=z,                
        colorscale='Viridis',   
        opacity=0.8
    )
)])

fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0), 
    scene={"aspectmode":"data"}
)

In [ ]:
# mesh = o3d.io.read_triangle_mesh("../data/Bspline_202311/aligned/maple_leaf.ply")
mesh = o3d.io.read_triangle_mesh("data/mesh/translate_"+name+".ply")


vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)
mesh

In [ ]:
x_edge,y_edge,z_edge = coord_3d[0:-1:100].T

x_surf, y_surf, z_surf = vertices.T

I, J, K = triangles.T

fig = go.Figure(data=[go.Scatter3d(
    x=x_edge,
    y=y_edge,
    z=z_edge,
    mode='markers',
    marker=dict(
        size=1,
        color=z,                
        colorscale='Viridis',   
        opacity=0.8
    )
),
    go.Mesh3d(
            x=x_surf,
            y=y_surf,
            z=z_surf, 
            i=I, 
            j=J, 
            k=K, 
            flatshading=True,
            # colorscale=colorscale, 
            intensity=z_surf, 
            showscale=False)
                     ])

fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0), 
    scene={"aspectmode":"data"}
)
fig.show()

In [ ]:
plt.hist(x_edge)

In [ ]:
plt.scatter(x_edge,y_edge+2)

## データ前処理

In [ ]:
pca = PCA(n_components=3)
coord_2d = pca.fit_transform(coord_3d)
coord_2d = np.array([coord_2d[:,0], coord_2d[:,2]]).T

In [ ]:
coord_2d.shape

In [ ]:
#x_temp, y_temp = coord_3d[:,0], coord_3d[:,1]+2
#coord_2d = np.array([x_temp, y_temp]).T

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.axis('equal')
sns.scatterplot(x=coord_2d[:,0], y=coord_2d[:,1], ax=ax)

In [ ]:
r, theta = cvt_polar(coord_2d[:,0], coord_2d[:,1])

# thetaは-Pi/2から3Pi/2までとする場合
# 開始，終了点の曲率を低くするためのハック
# theta = np.array([t+2*np.pi if t < -np.pi/2 else t for t in theta])

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.axis('equal')
sns.scatterplot(x=coord_2d[:,0], y=coord_2d[:,1], hue=theta, ax=ax)

## B-spline fitting 閉曲線版推定

In [ ]:
from scipy.optimize import curve_fit, least_squares, minimize
from scipy.interpolate import BSpline

In [ ]:
# def generate_knots_circular_bspline(start, end, n_interval, d):
    
#     step = (end-start)/n_interval
#     knots_ = [i for i in np.arange(start, end+step, step)]
#     knots_p = [knots_[i+len(knots_)-2] - (knots_[-1] - knots_[0]) for i in range(1-d, 1)]
#     knots_a = [knots_[i-len(knots_)]+ (knots_[-1]-knots_[0]) for i in range(len(knots_)+1,len(knots_)+d+2)]
    
#     knots = knots_p + knots_ + knots_a
    
#     return np.array(knots)

def generate_knots_circular_bspline(start, end, n_interval, d):
    
    step = (end-start)/n_interval
    knots_ = [i for i in np.arange(start, end, step)]
    knots_p = [knots_[i+len(knots_)-2] - (knots_[-1] - knots_[0]) for i in range(1-d, 1)]
    knots_a = [knots_[i-len(knots_)]+ (knots_[-1]-knots_[0]) for i in range(len(knots_)+1,len(knots_)+d+2)]
    
    knots = knots_p + knots_ + knots_a
    
    return np.array(knots)

In [ ]:
degree = 3
n_interval = 64
knots = generate_knots_circular_bspline(-np.pi, np.pi, n_interval, degree)
print(len(knots), knots)

# def bspline_wrapper(theta, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17, c18, c19):
#     bspline = BSpline(knots, 
#                       [c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17, c18, c19, 0], 
#                       degree, extrapolate="periodic")
#     return bspline(theta)

# def bspline_wrapper(theta, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17, c18, c19, c20):
#     bspline = BSpline(knots, 
#                       [c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17, c18, c19, c20], 
#                       degree, extrapolate="periodic")
#     return bspline(theta)
# def bspline_wrapper(theta, 
#                     c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, 
#                     c11, c12, c13, c14, c15, c16, c17, c18, c19, c20, 
#                     c21, c22, c23, c24, c25, c26, c27, c28, c29, c30,
#                     c31, c32, c33, c34, c35, c36, c37, c38, c39
#                    ):
#     bspline = BSpline(knots, 
#                       [c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, 
#                     c11, c12, c13, c14, c15, c16, c17, c18, c19, c20, 
#                     c21, c22, c23, c24, c25, c26, c27, c28, c29, c30,
#                     c31, c32, c33, c34, c35, c36, c37, c38, c39], 
#                       degree, extrapolate="periodic")
#     return bspline(theta)

def generate_bspline_wrapper(knots, degree):
    str_def = """
def bspline_wrapper(theta, {c}):
    bspline = BSpline({knots}, [{c}], {degree}, extrapolate="periodic")
    return bspline(theta)
"""
    
    n_c = len(knots)
    knots_str = np.array2string(knots, precision=10^4, max_line_width=np.inf,separator=",")
    c_str = ", ".join(["c"+str(i) for i in range(n_c)])
    exec(str_def.format(c=c_str,  knots = knots_str, degree=degree), globals())
    
    return bspline_wrapper

bspline_wrapper = generate_bspline_wrapper(knots, degree)
bspline_wrapper

In [ ]:
theta_r = theta + (10**-8)*np.random.random(len(theta))
idx_sorted = np.argsort(theta_r)
theta_sorted = theta_r[idx_sorted]
coord_3d_sorted = coord_3d[idx_sorted]

print("original: ", theta.shape, "sorted: ", np.unique(theta_sorted).shape)

In [ ]:
popt_x, pcov_x = curve_fit(bspline_wrapper, theta_sorted, coord_3d_sorted[:,0], bounds=(-50,50), ftol=10**-6)
print("done: x")
popt_y, pcov_y = curve_fit(bspline_wrapper, theta_sorted, coord_3d_sorted[:,1], bounds=(-50,50), ftol=10**-6)
print("done: y")
popt_z, pcov_z = curve_fit(bspline_wrapper, theta_sorted, coord_3d_sorted[:,2], bounds=(-50,50), ftol=10**-6)
print("done: z")

In [ ]:
popt_x

In [ ]:
pcov_x

In [ ]:
popt_y

In [ ]:
pcov_y

In [ ]:
popt_z

### 推定結果に基づく座標値

In [ ]:
theta_recon = np.linspace(-np.pi, np.pi, 3600)
coord_3d_recon = np.stack([
    bspline_wrapper(theta_recon, *popt_x), 
    bspline_wrapper(theta_recon, *popt_y), 
    bspline_wrapper(theta_recon, *popt_z), 
]).T

In [ ]:
coord_3d_recon[0]

In [ ]:
coord_3d_recon[-1]

In [ ]:
sns.scatterplot(x=theta_recon, y=coord_3d_recon[:,0])
sns.scatterplot(x=theta_recon, y=coord_3d_recon[:,1])
sns.scatterplot(x=theta_recon, y=coord_3d_recon[:,2])

In [ ]:
# FILE_PATH_COORD_RECON = "../data/Bspline_202311/leaf_lobed_nIntervals_64.csv"
# FILE_PATH_COEF = "../data/Bspline_202311/leaf_lobed_coef_nIntervals_64.csv"

FILE_PATH_COORD_RECON = "data/result/"+name+"_nIntervals_64.csv"
FILE_PATH_COEF = "data/result/"+name+"_coef_nIntervals_64.csv"

np.savetxt(FILE_PATH_COORD_RECON, coord_3d_recon)
pd.DataFrame(np.stack([popt_x, popt_y, popt_z],1), columns=["c_x", "c_y", "c_z"]).to_csv(FILE_PATH_COEF, index=False)

## 可視化

In [ ]:
x,y,z = coord_3d[0:-1:100].T

x_recon,y_recon,z_recon = coord_3d_recon.T
x_recon = np.append(x_recon, x_recon[0])
y_recon = np.append(y_recon, y_recon[0])
z_recon = np.append(z_recon, z_recon[0])

fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=1,
        color=z,                
        colorscale='Viridis',   
        opacity=0.8
    )
),go.Scatter3d(
    x=x_recon, y=y_recon, z=z_recon,
    marker=dict(
        size=1,
        colorscale='Viridis',
    ),
    line=dict(
        color="red",
        width=10
    )
)])

fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0), 
    scene={"aspectmode":"data"}
)

In [ ]:
x_surf, y_surf, z_surf = vertices.T
I, J, K = triangles.T

fig = go.Figure(data=[
    go.Mesh3d(
            x=x_surf,
            y=y_surf,
            z=z_surf, 
            i=I, 
            j=J, 
            k=K, 
            color="lime",
            # flatshading=True,
            # colorscale=colorscale, 
            # intensity=z_surf, 
            opacity=0.7,
            showscale=False)
                     ])

fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False)
fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0), 
    scene={"aspectmode":"data"},
    scene_camera = dict(eye=dict(x=-3, y=-3, z=3)),
    plot_bgcolor = "rgba(0, 0, 0, 0)",
    paper_bgcolor = "rgba(0, 0, 0, 0)",
)
fig.show()
fig.write_image("data/result/"+name+".png", scale=6)

In [ ]:
# curve_fragments[0][0]
len(curve_fragments[0])

In [ ]:
x_recon,y_recon,z_recon = coord_3d_recon.T
# x_recon = np.append(x_recon, x_recon[0])
# y_recon = np.append(y_recon, y_recon[0])
# z_recon = np.append(z_recon, z_recon[0])

x_surf, y_surf, z_surf = vertices.T
I, J, K = triangles.T

fig = go.Figure(data=[
    go.Mesh3d(
            x=x_surf,
            y=y_surf,
            z=z_surf, 
            i=I, 
            j=J, 
            k=K, 
            color="lime",
            # flatshading=True,
            # colorscale=colorscale, 
            # intensity=z_surf, 
            opacity=0.7,
            showscale=False),
    go.Scatter3d(
    x=x_recon,
    y=y_recon,
    z=z_recon,
    mode='lines',
    line=dict(
        width=12,
        color="purple",                
    )
)
                     ])

fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False)
fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0), 
    scene={"aspectmode":"data"},
    scene_camera = dict(eye=dict(x=-3, y=-3, z=3)),
    plot_bgcolor = "rgba(0, 0, 0, 0)",
    paper_bgcolor = "rgba(0, 0, 0, 0)",
)
fig.show()
fig.write_image("data/result/"+name+"_bspline.png", scale=6)

In [ ]:
x,y,z = coord_3d_recon.T

fig = go.Figure(data=[go.Scatter3d(
    x=x, y=y, z=z,
    marker=dict(
        size=1,
        colorscale='Viridis',
    ),
    line=dict(
        color="red",
        width=10
    )
)
                     ])

fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0), 
    scene={"aspectmode":"data"}
)

In [ ]:
x,y,z = coord_3d_recon.T

fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=2,
        color=theta_recon,                
        colorscale='Viridis',   
        opacity=0.8
    )
)])

fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0), 
    scene={"aspectmode":"data"}
)

## 差の計算
[Shapely](https://shapely.readthedocs.io/en/stable/)を使って，Fréchet距離を計算する

In [ ]:
from shapely import linearrings, frechet_distance, hausdorff_distance

In [ ]:
poly1 = linearrings(curve_list[0])
poly2 = linearrings(curve_list[1])

In [ ]:
frechet_distance(poly1, poly2)

In [ ]:
frechet_distance(poly2, poly1)

In [ ]:
hausdorff_distance(poly1, poly2)